In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.cluster import KMeans

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

from catboost import CatBoostRegressor
from catboost import CatBoostClassifier 

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
import warnings

# Установка параметров для игнорирования предупреждений
warnings.filterwarnings('ignore')

pd.set_option('display.width', 200)


In [2]:
# Загрузка данных
data = pd.read_csv('data\\train.csv')

In [3]:
# Создание экземпляра модели K-means
kmeans = KMeans(n_clusters=5, random_state=42)

# Обучение модели на не пропущенных значениях возраста
age_data = data[data['Age'].notnull()]['Age'].values.reshape(-1, 1)
kmeans.fit(age_data)

# Получение меток кластеров
data['AgeCluster'] = kmeans.predict(data['Age'].fillna(-1).values.reshape(-1, 1))

# Вычисление медианных значений возраста для каждого кластера
age_cluster_medians = data.groupby('AgeCluster')['Age'].median()

# Замена пропущенных значений возраста медианными значениями по соответствующему кластеру
data['Age'] = data.apply(lambda row: age_cluster_medians[row['AgeCluster']] if pd.isnull(row['Age']) else row['Age'], axis=1)

# Удаление столбца с метками кластеров
data.drop('AgeCluster', axis=1, inplace=True)


In [4]:
# Находим самое частое значение в столбце 'Embarked'
most_common_embarked = data['Embarked'].mode()[0]

# Заменяем пропущенные значения в столбце 'Embarked' на самое частое значение
data['Embarked'].fillna(most_common_embarked, inplace=True)

In [5]:
# Заменим значения кабины на ближайщее среднее значение цены билета

# Условие для замены пропусков в зависимости от класса
condition_1 = (data['Pclass'] == 1) & (data['Cabin'].isnull())
condition_2 = (data['Pclass'] == 2) & (data['Cabin'].isnull())
condition_3 = (data['Pclass'] == 3) & (data['Cabin'].isnull())

# Замена пропусков в столбце "Cabin" в зависимости от класса
data.loc[condition_1, 'Cabin'] = 'D'
data.loc[condition_2, 'Cabin'] = 'F'
data.loc[condition_3, 'Cabin'] = 'G'
# Убираем все символы после первой буквы в столбце Cabin
data['Cabin'] = data['Cabin'].str[0]

In [6]:
# Преобразование категориальных переменных в числовые методом get_dummies
data = pd.get_dummies(data, columns=['Embarked', 'Sex', 'Cabin'], drop_first=True)

In [7]:
# Создаем объект LabelEncoder
label_encoder = LabelEncoder()

# Преобразуем столбец 'Name'
data['Name'] = label_encoder.fit_transform(data['Name'])

# Преобразуем столбец 'Ticket'
data['Ticket'] = label_encoder.fit_transform(data['Ticket'])
# Инициализация объекта MinMaxScaler
scaler = MinMaxScaler()

# Выбор числовых признаков, которые нужно масштабировать
numeric_features = ['Age', 'Fare', 'Ticket', 'Name', 'Pclass', 'SibSp', 'Parch']

# Масштабирование выбранных признаков
data[numeric_features] = scaler.fit_transform(data[numeric_features])

In [8]:
# Поиск столбцов с пропущенными значениями
columns_with_missing_values = data.columns[data.isnull().any()]

if len(columns_with_missing_values) > 0:
    print("Столбцы с пропущенными значениями:")
    for column in columns_with_missing_values:
        print(column)
else:
    print("Пропущенные значения не обнаружены.")

Пропущенные значения не обнаружены.


## RandomForestClassifier 

In [9]:
# Признаки (X) и целевая переменная (y)
X = data.drop(['Survived'], axis=1)
y = data['Survived']


In [10]:

# Разделение на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создание модели случайного леса
rf_model = RandomForestClassifier(random_state=42)

# Обучение модели на тренировочных данных
rf_model.fit(X_train, y_train)

# Получение предсказаний на тестовом наборе
y_pred = rf_model.predict(X_test)

# Оценка точности модели
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8324022346368715


In [11]:
# Загрузка тестовых данных
test = pd.read_csv('data\\test.csv')
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [12]:
# Создание экземпляра модели K-means
kmeans = KMeans(n_clusters=5, random_state=42)

# Обучение модели на не пропущенных значениях возраста
age_data = data[data['Age'].notnull()]['Age'].values.reshape(-1, 1)
kmeans.fit(age_data)

# Получение меток кластеров
data['AgeCluster'] = kmeans.predict(data['Age'].fillna(-1).values.reshape(-1, 1))

# Вычисление медианных значений возраста для каждого кластера
age_cluster_medians = data.groupby('AgeCluster')['Age'].median()

# Замена пропущенных значений возраста медианными значениями по соответствующему кластеру
data['Age'] = data.apply(lambda row: age_cluster_medians[row['AgeCluster']] if pd.isnull(row['Age']) else row['Age'], axis=1)

# Удаление столбца с метками кластеров
data.drop('AgeCluster', axis=1, inplace=True)

In [13]:
# Находим самое частое значение в столбце 'Embarked'
most_common_embarked = test['Embarked'].mode()[0]

# Заменяем пропущенные значения в столбце 'Embarked' на самое частое значение
test['Embarked'].fillna(most_common_embarked, inplace=True)

In [14]:
# Заменим значения кабины на ближайщее среднее значение цены билета

# Условие для замены пропусков в зависимости от класса
condition_1 = (test['Pclass'] == 1) & (test['Cabin'].isnull())
condition_2 = (test['Pclass'] == 2) & (test['Cabin'].isnull())
condition_3 = (test['Pclass'] == 3) & (test['Cabin'].isnull())

# Замена пропусков в столбце "Cabin" в зависимости от класса
test.loc[condition_1, 'Cabin'] = 'D'
test.loc[condition_2, 'Cabin'] = 'F'
test.loc[condition_3, 'Cabin'] = 'G'
# Убираем все символы после первой буквы в столбце Cabin
test['Cabin'] = test['Cabin'].str[0]

In [15]:
# Преобразование категориальных переменных в числовые методом get_dummies
test = pd.get_dummies(test, columns=['Embarked', 'Sex', 'Cabin'], drop_first=True)


In [16]:
# Создаем объект LabelEncoder
label_encoder = LabelEncoder()

# Преобразуем столбец 'Name'
test['Name'] = label_encoder.fit_transform(test['Name'])

# Преобразуем столбец 'Ticket'
test['Ticket'] = label_encoder.fit_transform(test['Ticket'])
# Инициализация объекта MinMaxScaler
scaler = MinMaxScaler()

# Выбор числовых признаков, которые нужно масштабировать
numeric_features = ['Age', 'Fare', 'Ticket', 'Name', 'Pclass', 'SibSp', 'Parch']

# Масштабирование выбранных признаков
test[numeric_features] = scaler.fit_transform(test[numeric_features])

In [17]:
# Определение различий в столбцах между двумя наборами данных
extra_feature = set(X.columns) - set(test.columns)

# Удаление лишнего признака из DataFrame data, если такой есть
if extra_feature:
    X.drop(columns=list(extra_feature), inplace=True)
    print(f"Удален лишний признак из DataFrame X: {extra_feature}")
else:
    print("Не обнаружено лишних признаков в DataFrame X.")

# Проверка наличия дополнительных признаков в DataFrame test
extra_feature_test = set(test.columns) - set(X.columns)
if extra_feature_test:
    print(f"Дополнительные признаки в DataFrame test: {extra_feature_test}")
else:
    print("Не обнаружено дополнительных признаков в DataFrame test.")


Удален лишний признак из DataFrame X: {'Cabin_T'}
Не обнаружено дополнительных признаков в DataFrame test.


In [18]:
test.head()

,PassengerId,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Embarked_Q,Embarked_S,Sex_male,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G
0,892,1.0,0.494005,0.452723,0.000,0.000000,0.419890,0.015282,True,False,True,False,False,False,False,False,True
1,893,1.0,0.966427,0.617566,0.125,0.000000,0.610497,0.013663,False,True,False,False,False,False,False,False,True
2,894,0.5,0.645084,0.815377,0.000,0.000000,0.201657,0.018909,True,False,True,False,False,False,False,True,False
3,895,1.0,0.978417,0.353818,0.000,0.000000,0.406077,0.016908,False,True,True,False,False,False,False,False,True
4,896,1.0,0.426859,0.287881,0.125,0.111111,0.381215,0.023984,False,True,False,False,False,False,False,False,True


In [19]:
# Задаем сетку параметров для перебора
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Создаем модель случайного леса
rf_model = RandomForestClassifier(random_state=42)

# Инициализируем Grid Search с кросс-валидацией
grid_search = GridSearchCV(rf_model, param_grid, cv=5, scoring='accuracy')

# Запускаем поиск по сетке параметров
grid_search.fit(X_train, y_train)

# Выводим лучшие параметры
print("Лучшие параметры:", grid_search.best_params_)

# Получаем предсказания на тестовом наборе с лучшими параметрами
y_pred = grid_search.predict(X_test)

# Оцениваем точность модели
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Лучшие параметры: {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100}
Accuracy: 0.8379888268156425


In [21]:
# Запускаем поиск по сетке параметров
grid_search.fit(X, y)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_depth': [None, 10, 20],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring='accuracy')

In [22]:
predictions = grid_search.predict(test)

# Создание DataFrame с предсказаниями
submission = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': predictions})

# Сохранение предсказаний в файл
submission.to_csv('gender_submission.csv', index=False)